
![Image Name](https://cdn.kesci.com/upload/image/prl4xjl1zc.png?imageView2/0/w/960/h/960)


#### 1、如何用python 快速处理数据？
* Python处理数据利器 → Pandas
* 数据一般格式：csv/xlsx
* 如何用pandas读取数据
* 案例：用pandas处理商铺数据

1）老方法：列表字典

In [2]:
# 数据读取

f = open('/home/kesci/商铺数据.csv','r',encoding='utf8')
for i in f.readlines()[:5]:
    print(i.split(','))  
    # 查看数据

['\ufeffclassify', 'name', 'comment', 'star', 'price', 'address', 'commentlist\n']
['美食', '望蓉城老坛酸菜鱼(合生汇店)', '我要点评', '该商户暂无星级', '人均                                        ￥125', '翔殷路1099号合生汇5F', '口味8.3                                环境8.4                                服务8.5\n']
['美食', '泰国街边料理', '74                    条点评', '准四星商户', '人均                                        ￥48', '黄兴路合生汇B2美食集市内', '口味7.4                                环境7.6                                服务7.4\n']
['美食', '壹面如故(苏宁生活广场店)', '265                    条点评', '准四星商户', '人均                                        ￥21', '邯郸路585号苏宁生活广场B1层', '口味7.0                                环境7.2                                服务7.2\n']
['美食', '鮨谷•Aburiya(合生汇店)', '2748                    条点评', '准五星商户', '人均                                        ￥142', '翔殷路1099号合生广场5楼23、28铺位', '口味8.9                                环境8.5                                服务8.4\n']


In [3]:
# 清洗字段

# 创建comment、price、commentlist清洗函数
# 函数式编程

def fcm(s):
    if '条' in s:
        return int(s.split(' ')[0])
    else:
        return '缺失数据'
# comment清洗函数：用空格分段，选取结果list的第一个为点评数，并且转化为整型

def fpr(s):
    if '￥' in s:
        return float(s.split('￥')[-1])
    else:
        return  '缺失数据'
# print清洗函数：用￥分段，选取结果list的最后一个为人均价格，并且转化为浮点型

def fcl(s):
    if ' ' in s:
        quality = float(s.split('                                ')[0][2:])
        environment = float(s.split('                                ')[1][2:])
        service = float(s.split('                                ')[2][2:-1])
        return [quality,environment,service]
    else:
        return  '缺失数据'
# commentlist清洗函数：用空格分段，分别清洗出质量、环境及服务数据，并转化为浮点型

In [4]:
# 数据清洗

datalst = []  # 创建空列表

f.seek(0)
n = 0  # 创建计数变量
for i in f.readlines()[1:10]:
    data = i.split(',')
    #print(data)
    classify = data[0]             # 提取分类
    name = data[1]                 # 提取店铺名称
    comment_count = fcm(data[2])   # 提取评论数量
    star = data[3]                 # 提取星级
    price = fpr(data[4])           # 提取人均
    add = data[5]                  # 提取地址
    qua = fcl(data[6])[0]          # 提取质量评分
    env = fcl(data[6])[1]          # 提取环境评分
    ser = fcl(data[6])[2]          # 提取服务评分
    if  '缺失数据' not in [comment_count, price, qua]:   # 用于判断是否有数据缺失
        n += 1
        data_re = [['classify',classify],
                  ['name',name],
                  ['comment_count',comment_count],
                  ['star',star],
                  ['price',price],
                  ['address',add],
                  ['quality',qua],
                  ['environment',env],
                  ['service',ser]]
        datalst.append(dict(data_re))   # 生成字典，并存入列表datalst
        print('成功加载%i条数据' %n)
    else:
        continue
    
print(datalst) 
print('总共加载%i条数据' %n)

成功加载1条数据
成功加载2条数据
成功加载3条数据
成功加载4条数据
成功加载5条数据
成功加载6条数据
成功加载7条数据
[{'classify': '美食', 'name': '泰国街边料理', 'comment_count': 74, 'star': '准四星商户', 'price': 48.0, 'address': '黄兴路合生汇B2美食集市内', 'quality': 7.4, 'environment': 7.6, 'service': 7.4}, {'classify': '美食', 'name': '壹面如故(苏宁生活广场店)', 'comment_count': 265, 'star': '准四星商户', 'price': 21.0, 'address': '邯郸路585号苏宁生活广场B1层', 'quality': 7.0, 'environment': 7.2, 'service': 7.2}, {'classify': '美食', 'name': '鮨谷•Aburiya(合生汇店)', 'comment_count': 2748, 'star': '准五星商户', 'price': 142.0, 'address': '翔殷路1099号合生广场5楼23、28铺位', 'quality': 8.9, 'environment': 8.5, 'service': 8.4}, {'classify': '美食', 'name': '麦当劳(万达店)', 'comment_count': 785, 'star': '准四星商户', 'price': 24.0, 'address': '邯郸路600号万达商业广场B1楼A05号铺', 'quality': 7.4, 'environment': 7.2, 'service': 7.2}, {'classify': '美食', 'name': '蒸年青STEAMYOUNG(百联又一城购物中心店)', 'comment_count': 3779, 'star': '准五星商户', 'price': 70.0, 'address': '淞沪路8号百联又一城购物中心7层', 'quality': 8.6, 'environment': 8.6, 'service': 8.6}, {'classify': '

2）新方法：用pandas处理

In [51]:
import pandas as pd 
    # 导入pandas模块

import warnings
warnings.filterwarnings('ignore') 
    # 不发出警告

print('成功导入模块')

成功导入模块


In [6]:
# 如何用pandas读取数据 - csv

df = pd.read_csv('/home/kesci/商铺数据.csv')
print(type(df),df['name'].dtype) # 查看df类型，查看df中一列的数值类型
df.head()

<class 'pandas.core.frame.DataFrame'> object


,classify,name,comment,star,price,address,commentlist
0,美食,望蓉城老坛酸菜鱼(合生汇店),我要点评,该商户暂无星级,人均 ￥125,翔殷路1099号合生汇5F,口味8.3 环境8.4 ...
1,美食,泰国街边料理,74 条点评,准四星商户,人均 ￥48,黄兴路合生汇B2美食集市内,口味7.4 环境7.6 ...
2,美食,壹面如故(苏宁生活广场店),265 条点评,准四星商户,人均 ￥21,邯郸路585号苏宁生活广场B1层,口味7.0 环境7.2 ...
3,美食,鮨谷•Aburiya(合生汇店),2748 条点评,准五星商户,人均 ￥142,翔殷路1099号合生广场5楼23、28铺位,口味8.9 环境8.5 ...
4,美食,我们的烤肉我们的馕,5 条点评,准四星商户,人均 -,邯郸路399-D3号,口味7.5 环境6.8 ...


In [7]:
# 用pandas处理商铺数据 - comment字段清洗

df1 = df[df['comment'].str.contains('条')]
df1['comment'] = df1['comment'].str.split('条').str[0]
print(df1['comment'].dtype)

df1['comment'] = df1['comment'].astype('int')
print(df1['comment'].dtype) # 更改列数值类型

df1.head()

object
int64


,classify,name,comment,star,price,address,commentlist
1,美食,泰国街边料理,74,准四星商户,人均 ￥48,黄兴路合生汇B2美食集市内,口味7.4 环境7.6 ...
2,美食,壹面如故(苏宁生活广场店),265,准四星商户,人均 ￥21,邯郸路585号苏宁生活广场B1层,口味7.0 环境7.2 ...
3,美食,鮨谷•Aburiya(合生汇店),2748,准五星商户,人均 ￥142,翔殷路1099号合生广场5楼23、28铺位,口味8.9 环境8.5 ...
4,美食,我们的烤肉我们的馕,5,准四星商户,人均 -,邯郸路399-D3号,口味7.5 环境6.8 ...
5,美食,麦当劳(万达店),785,准四星商户,人均 ￥24,邯郸路600号万达商业广场B1楼A05号铺,口味7.4 环境7.2 ...


In [8]:
# 用pandas处理商铺数据 - price字段清洗

df1 = df1[df1['price'].str.contains('￥')]
df1['price'] = df1['price'].str.split('￥').str[-1]

df1['price'] = df1['price'].astype('float')
print(df1['price'].dtype) # 更改列数值类型

df1.head()

float64


,classify,name,comment,star,price,address,commentlist
1,美食,泰国街边料理,74,准四星商户,48.0,黄兴路合生汇B2美食集市内,口味7.4 环境7.6 ...
2,美食,壹面如故(苏宁生活广场店),265,准四星商户,21.0,邯郸路585号苏宁生活广场B1层,口味7.0 环境7.2 ...
3,美食,鮨谷•Aburiya(合生汇店),2748,准五星商户,142.0,翔殷路1099号合生广场5楼23、28铺位,口味8.9 环境8.5 ...
5,美食,麦当劳(万达店),785,准四星商户,24.0,邯郸路600号万达商业广场B1楼A05号铺,口味7.4 环境7.2 ...
6,美食,蒸年青STEAMYOUNG(百联又一城购物中心店),3779,准五星商户,70.0,淞沪路8号百联又一城购物中心7层,口味8.6 环境8.6 ...


In [9]:
# 用pandas处理商铺数据 - commentlist

df1['quality'] = df1['commentlist'].str.split('                                ').str[0].str[2:].astype('float')
df1['environment'] = df1['commentlist'].str.split('                                ').str[1].str[2:].astype('float')
df1['service'] = df1['commentlist'].str.split('                                ').str[2].str[2:].astype('float')
# 字段清洗

del df1['commentlist']

df1.head()

,classify,name,comment,star,price,address,quality,environment,service
1,美食,泰国街边料理,74,准四星商户,48.0,黄兴路合生汇B2美食集市内,7.4,7.6,7.4
2,美食,壹面如故(苏宁生活广场店),265,准四星商户,21.0,邯郸路585号苏宁生活广场B1层,7.0,7.2,7.2
3,美食,鮨谷•Aburiya(合生汇店),2748,准五星商户,142.0,翔殷路1099号合生广场5楼23、28铺位,8.9,8.5,8.4
5,美食,麦当劳(万达店),785,准四星商户,24.0,邯郸路600号万达商业广场B1楼A05号铺,7.4,7.2,7.2
6,美食,蒸年青STEAMYOUNG(百联又一城购物中心店),3779,准五星商户,70.0,淞沪路8号百联又一城购物中心7层,8.6,8.6,8.6


#### 2、初学者在 数据索引的常见错误
* 列索引/行索引问题： df[…] / .iloc / .loc 
* 布尔型索引

In [10]:
# 列索引 df[...] - 单列索引，结果series

df1['quality'].head()

1    7.4
2    7.0
3    8.9
5    7.4
6    8.6
Name: quality, dtype: float64

In [11]:
# 列索引 df[...] - 多列索引，结果dataframe

df1[['quality','environment','service']].head()

,quality,environment,service
1,7.4,7.6,7.4
2,7.0,7.2,7.2
3,8.9,8.5,8.4
5,7.4,7.2,7.2
6,8.6,8.6,8.6


In [12]:
# 行索引 - loc
# 按照index选择行，只选择一行输出Series，选择多行输出Dataframe

import numpy as np

df = pd.DataFrame(np.random.rand(12).reshape(3,4)*100,
                   index = ['one','two','three'],
                   columns = ['a','b','c','d'])

data1 = df.loc['one']
data2 = df.loc[['one','two']]
print(data1,'\n-----',type(data1))
print(data2,'\n-----',type(data2))

a    94.697126
b    68.728959
c    86.539632
d     7.784895
Name: one, dtype: float64 
----- <class 'pandas.core.series.Series'>
             a          b          c         d
one  94.697126  68.728959  86.539632  7.784895
two  52.405708  53.614338  66.878720  1.469412 
----- <class 'pandas.core.frame.DataFrame'>


In [13]:
# 行索引 - iloc
# 按照整数位置（从轴的0到length-1）选择行

data3 = df.iloc[0]
data4 = df.iloc[[0,2]]
print(data3,'\n-----',type(data3))
print(data4,'\n-----',type(data4))

a    94.697126
b    68.728959
c    86.539632
d     7.784895
Name: one, dtype: float64 
----- <class 'pandas.core.series.Series'>
               a          b          c          d
one    94.697126  68.728959  86.539632   7.784895
three  68.599077  93.439114  99.715899  94.336584 
----- <class 'pandas.core.frame.DataFrame'>


In [14]:
# 布尔型索引

df1[df1['comment'] >50].head()

,classify,name,comment,star,price,address,quality,environment,service
1,美食,泰国街边料理,74,准四星商户,48.0,黄兴路合生汇B2美食集市内,7.4,7.6,7.4
2,美食,壹面如故(苏宁生活广场店),265,准四星商户,21.0,邯郸路585号苏宁生活广场B1层,7.0,7.2,7.2
3,美食,鮨谷•Aburiya(合生汇店),2748,准五星商户,142.0,翔殷路1099号合生广场5楼23、28铺位,8.9,8.5,8.4
5,美食,麦当劳(万达店),785,准四星商户,24.0,邯郸路600号万达商业广场B1楼A05号铺,7.4,7.2,7.2
6,美食,蒸年青STEAMYOUNG(百联又一城购物中心店),3779,准五星商户,70.0,淞沪路8号百联又一城购物中心7层,8.6,8.6,8.6


In [15]:
# 布尔型索引

df1[df1['comment'] >50][['quality','environment','service']].head()

,quality,environment,service
1,7.4,7.6,7.4
2,7.0,7.2,7.2
3,8.9,8.5,8.4
5,7.4,7.2,7.2
6,8.6,8.6,8.6


#### 3、Pandas套路总结
* 数据类型问题，series和dataframe
* 去重
* 替换
* 分组
* 合并

In [16]:
# Series，带有标签的一维数组，可以保存任何数据类型（整数，字符串，浮点数，Python对象等）,轴标签统称为索引
s = pd.Series(np.random.rand(5))
print(s)
print(type(s))
    # 查看数据、数据类型

0    0.177446
1    0.620665
2    0.448813
3    0.304528
4    0.834865
dtype: float64
<class 'pandas.core.series.Series'>


In [17]:
# Dataframe 数据结构
# Dataframe是一个表格型的数据结构，“带有标签的二维数组”。
# Dataframe带有index（行标签）和columns（列标签）

data = {'name':['Jack','Tom','Mary'],
        'age':[18,19,20],
       'gender':['m','m','w']}
frame = pd.DataFrame(data)
print(frame)  
print(type(frame))
print(frame.index,'\n该数据类型为：',type(frame.index))
print(frame.columns,'\n该数据类型为：',type(frame.columns))
print(frame.values,'\n该数据类型为：',type(frame.values))
    # 查看数据，数据类型为dataframe
    # .index查看行标签
    # .columns查看列标签
    # .values查看值，数据类型为ndarray

   name  age gender
0  Jack   18      m
1   Tom   19      m
2  Mary   20      w
<class 'pandas.core.frame.DataFrame'>
RangeIndex(start=0, stop=3, step=1) 
该数据类型为： <class 'pandas.core.indexes.range.RangeIndex'>
Index(['name', 'age', 'gender'], dtype='object') 
该数据类型为： <class 'pandas.core.indexes.base.Index'>
[['Jack' 18 'm']
 ['Tom' 19 'm']
 ['Mary' 20 'w']] 
该数据类型为： <class 'numpy.ndarray'>


In [18]:
# 常用创建方法

# 1、直接读取数据，pd.read...
# 2、结合numpy创建随机数，常用方法 → 字典创建dataframe / 随机数+columns参数设置

df = pd.DataFrame({'value1':np.random.randn(10)*10,
                  'value2':np.random.randn(10)*100})
df

,value1,value2
0,4.175609,30.933675
1,7.503537,-163.532084
2,6.167065,108.163448
3,23.570007,-83.975524
4,-2.161862,-79.798933
5,-7.508486,26.962803
6,12.331537,35.163374
7,7.887318,81.110088
8,-3.522070,-142.800625
9,11.186132,100.173304


In [19]:
df = pd.DataFrame(np.random.randn(10,2)*100,columns = ['value1','value2'])
df

,value1,value2
0,-63.206988,-2.150619
1,125.237838,-45.590565
2,-67.146351,-58.219103
3,64.619886,-79.976598
4,-131.689060,-13.917292
5,55.994208,126.769870
6,-107.287142,-48.974914
7,-83.622731,17.325343
8,-165.669454,9.548235
9,16.626523,-120.595207


In [20]:
# 去重 .duplicated

s = pd.Series([1,1,1,1,2,2,2,3,4,5,5,5,5])
print(s.duplicated())
print(s[s.duplicated() == False])
print('-----')
    # 判断是否重复
    # 通过布尔判断，得到不重复的值
    
s_re = s.drop_duplicates()
print(s_re)
print('-----')
    # drop.duplicates移除重复
    # inplace参数：是否替换原值，默认False

df = pd.DataFrame({'key1':['a','a',3,4,5],
                  'key2':['a','a','b','b','c']})
print(df.duplicated())
print(df['key2'].duplicated())
    # Dataframe中使用duplicated

0     False
1      True
2      True
3      True
4     False
5      True
6      True
7     False
8     False
9     False
10     True
11     True
12     True
dtype: bool
0    1
4    2
7    3
8    4
9    5
dtype: int64
-----
0    1
4    2
7    3
8    4
9    5
dtype: int64
-----
0    False
1     True
2    False
3    False
4    False
dtype: bool
0    False
1     True
2    False
3     True
4    False
Name: key2, dtype: bool


In [21]:
# 替换 .replace

s = pd.Series(list('ascaazsd'))
print(s.replace('a', np.nan))
print(s.replace(['a','s'] ,np.nan))
print(s.replace({'a':'hello world!','s':123}))
# 可一次性替换一个值或多个值
# 可传入列表或字典

0    NaN
1      s
2      c
3    NaN
4    NaN
5      z
6      s
7      d
dtype: object
0    NaN
1    NaN
2      c
3    NaN
4    NaN
5      z
6    NaN
7      d
dtype: object
0    hello world!
1             123
2               c
3    hello world!
4    hello world!
5               z
6             123
7               d
dtype: object


In [22]:
# 分组

df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar','foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                   'D' : np.random.randn(8)})
print(df)
print('------')

print(df.groupby('A'), type(df.groupby('A')))
print('------')
# 直接分组得到一个groupby对象，是一个中间数据，没有进行计算

     A      B         C         D
0  foo    one -0.841533  1.915782
1  bar    one -0.153094  0.949085
2  foo    two  0.617556 -0.148972
3  bar  three -0.118922  1.054242
4  foo    two  0.996031  0.127400
5  bar    two -0.479970  1.104311
6  foo    one  0.273857  1.407518
7  foo  three -0.868798  0.400942
------
<pandas.core.groupby.groupby.DataFrameGroupBy object at 0x7f3f80562a20> <class 'pandas.core.groupby.groupby.DataFrameGroupBy'>
------


In [23]:
a = df.groupby('A').mean()
print(a,type(a),'\n',a.columns)
    # 通过分组后的计算，得到一个新的dataframe
    # 默认axis = 0，以行来分组
    # 可单个或多个（[]）列分组

            C         D
A                      
bar -0.250662  1.035879
foo  0.035423  0.740534 <class 'pandas.core.frame.DataFrame'> 
 Index(['C', 'D'], dtype='object')


In [25]:
# merge合并 → 类似excel的vlookup

df1 = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})
df2 = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
df3 = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                    'key2': ['K0', 'K1', 'K0', 'K1'],
                    'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3']})
df4 = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                    'key2': ['K0', 'K0', 'K0', 'K0'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']})
print(pd.merge(df1, df2, on='key'))
print('------')
    # left：第一个df
    # right：第二个df
    # on：参考键
    # 参数 left_on, right_on, left_index, right_index → 当键不为一个列时，可以单独设置左键与右键
    
print(pd.merge(df3, df4, on=['key1','key2']))
# 多个链接键

print(pd.merge(df3, df4,on=['key1','key2'], how = 'inner'))  
print('------')
    # 参数how → 合并方式
    # inner：默认，取交集
    # outer：取并集，数据缺失范围NaN
    # left：按照df3为参考合并，数据缺失范围NaN
    # right：按照df4为参考合并，数据缺失范围NaN

  key   A   B   C   D
0  K0  A0  B0  C0  D0
1  K1  A1  B1  C1  D1
2  K2  A2  B2  C2  D2
3  K3  A3  B3  C3  D3
------
  key1 key2   A   B   C   D
0   K0   K0  A0  B0  C0  D0
1   K1   K0  A2  B2  C1  D1
2   K1   K0  A2  B2  C2  D2


#### 预告，讲讲制图

In [34]:
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.DataFrame(np.random.randn(300, 4),columns=list('ABCD'))
df = df.cumsum()
df.plot(style = '--.',alpha = 0.8,colormap = 'GnBu',figsize=(12,6),grid=True)

<Figure size 864x432 with 1 Axes>

In [40]:
# 堆叠直方图

plt.figure(num=1)
df = pd.DataFrame({'a': np.random.randn(1000) + 1, 'b': np.random.randn(1000),
                    'c': np.random.randn(1000) - 1, 'd': np.random.randn(1000)-2},
                   columns=['a', 'b', 'c','d'])
df.plot.hist(stacked=True,
             bins=20,
             colormap='Greens_r',
             alpha=0.5,
             grid=True,
             figsize = (12,6))
    # 使用DataFrame.plot.hist()和Series.plot.hist()方法绘制
    # stacked：是否堆叠

<Figure size 432x288 with 0 Axes>

<Figure size 864x432 with 1 Axes>

In [41]:
# 生成多个直方图
df.hist(bins=50,figsize = (12,6))

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f3f46882588>,
      dtype=object)

<Figure size 864x432 with 4 Axes>

In [50]:
# plt.scatter()散点图

plt.figure(figsize=(8,6))
x = np.random.randn(1000)
y = np.random.randn(1000)
plt.figure(figsize = (12,6))
plt.scatter(x,y,
            marker='.',linewidth=0.8,edgecolor='k',
            s = np.random.randn(1000)*100,
            cmap = 'Reds',
            c = y,
            alpha = 0.8)
plt.grid()
    # s：散点的大小
    # c：散点的颜色
    # vmin,vmax：亮度设置，标量
    # cmap：colormap

<Figure size 576x432 with 0 Axes>

<Figure size 864x432 with 1 Axes>